In [2]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [3]:
from hyperopt import STATUS_OK, Trials, hp, space_eval, tpe, fmin
import lightgbm as lgb
from matplotlib import pyplot as plt
from matplotlib import rcParams
import numpy as np
from pathlib import Path
import pandas as pd
from sklearn.metrics import accuracy_score
from sklearn.model_selection import StratifiedKFold, train_test_split
import seaborn as sns
import warnings

In [4]:
rcParams['figure.figsize'] = (16, 8)
plt.style.use('fivethirtyeight')
pd.set_option('max_columns', 100)
pd.set_option("display.precision", 4)
warnings.simplefilter('ignore')

In [5]:

data_dir = Path('/content/drive/MyDrive/dacon/input')
feature_dir = Path('../build/feature')
val_dir = Path('/content/drive/MyDrive/dacon/build/val')
tst_dir = Path('/content/drive/MyDrive/dacon/build/tst')
sub_dir = Path('/content/drive/MyDrive/dacon/build/sub')

trn_file = data_dir / 'train.csv'
tst_file = data_dir / 'test.csv'
sample_file = data_dir / 'sample_submission.csv'

target_col = 'author'
n_fold = 5
n_class = 5
seed = 42

In [6]:
algo_name = 'lgbcv'
feature_name = 'stacking1-hyperopt'
model_name = f'{algo_name}_{feature_name}'

feature_file = feature_dir / f'{feature_name}.csv'
p_val_file = val_dir / f'{model_name}.val.csv'
p_tst_file = tst_dir / f'{model_name}.tst.csv'
sub_file = sub_dir / f'{model_name}.csv'

In [41]:
model_names = ['bla_emb','gru_emb','xgboost_feature','cnn_emb','rf_feature', 'lgbcv_feature', 'mta_emb', 'lr_tfidf', 
               'lstm_emb','svm_feature', 'gNB_feature','knn_feature','cnn_emb2', 'cnn_emb3', 'cnn_emb4','lstm_emb2',
               'gru_emb2','cnn_emb5','cnn_emb6','cnn_emb7','cnn_emb8','lstm_emb2','lstm_emb3', 'lgbcv_feature2']
trn = []
tst = []
feature_names = []
for model in model_names:
    trn.append(np.loadtxt(val_dir / f'{model}.val.csv', delimiter=','))
    tst.append(np.loadtxt(tst_dir / f'{model}.tst.csv', delimiter=','))
    feature_names += [f'{model}_class0', f'{model}_class1', f'{model}_class2', f'{model}_class3', f'{model}_class4']
    
trn = np.hstack(trn)
tst = np.hstack(tst)
feature_names

['bla_emb_class0',
 'bla_emb_class1',
 'bla_emb_class2',
 'bla_emb_class3',
 'bla_emb_class4',
 'gru_emb_class0',
 'gru_emb_class1',
 'gru_emb_class2',
 'gru_emb_class3',
 'gru_emb_class4',
 'xgboost_feature_class0',
 'xgboost_feature_class1',
 'xgboost_feature_class2',
 'xgboost_feature_class3',
 'xgboost_feature_class4',
 'cnn_emb_class0',
 'cnn_emb_class1',
 'cnn_emb_class2',
 'cnn_emb_class3',
 'cnn_emb_class4',
 'rf_feature_class0',
 'rf_feature_class1',
 'rf_feature_class2',
 'rf_feature_class3',
 'rf_feature_class4',
 'lgbcv_feature_class0',
 'lgbcv_feature_class1',
 'lgbcv_feature_class2',
 'lgbcv_feature_class3',
 'lgbcv_feature_class4',
 'mta_emb_class0',
 'mta_emb_class1',
 'mta_emb_class2',
 'mta_emb_class3',
 'mta_emb_class4',
 'lr_tfidf_class0',
 'lr_tfidf_class1',
 'lr_tfidf_class2',
 'lr_tfidf_class3',
 'lr_tfidf_class4',
 'lstm_emb_class0',
 'lstm_emb_class1',
 'lstm_emb_class2',
 'lstm_emb_class3',
 'lstm_emb_class4',
 'svm_feature_class0',
 'svm_feature_class1',
 'sv

In [42]:
y = pd.read_csv(trn_file, index_col=0, usecols=['index', target_col]).values.flatten()
print(y.shape, trn.shape, tst.shape)

(54879,) (54879, 120) (19617, 120)


In [43]:
X_trn, X_val, y_trn, y_val = train_test_split(trn, y, test_size=.2, random_state=seed)

In [44]:
params = {
    "objective": "multiclass",
    "n_estimators": 1000,
    "subsample_freq": 1,
    "random_state": seed,
    "n_jobs": -1,
}

space = {
    "learning_rate": hp.loguniform("learning_rate", np.log(0.01), np.log(0.3)),
    "num_leaves": hp.choice("num_leaves", [15, 31, 63, 127]),
    "colsample_bytree": hp.quniform("colsample_bytree", .5, .9, 0.1),
    "subsample": hp.quniform("subsample", .5, .9, 0.1),
    "min_child_samples": hp.choice('min_child_samples', [10, 25, 100])
}

In [45]:
def objective(hyperparams):
    model = lgb.LGBMClassifier(**params, **hyperparams)
    model.fit(X=X_trn, y=y_trn,
              eval_set=[(X_val, y_val)],
              eval_metric="multi_logloss",
              early_stopping_rounds=10,
              verbose=False)
    score = model.best_score_["valid_0"]["multi_logloss"]

    return {'loss': score, 'status': STATUS_OK, 'model': model}

trials = Trials()
best = fmin(fn=objective, space=space, trials=trials,
            algo=tpe.suggest, max_evals=10, verbose=1)

hyperparams = space_eval(space, best)
n_best = trials.best_trial['result']['model'].best_iteration_
params.update(hyperparams)
print(params)

100%|██████████| 10/10 [15:05<00:00, 90.56s/it, best loss: 0.41782910073593305]
{'objective': 'multiclass', 'n_estimators': 1000, 'subsample_freq': 1, 'random_state': 42, 'n_jobs': -1, 'colsample_bytree': 0.7000000000000001, 'learning_rate': 0.06608160290770859, 'min_child_samples': 25, 'num_leaves': 15, 'subsample': 0.7000000000000001}


In [46]:
cv = StratifiedKFold(n_splits=n_fold, shuffle=True, random_state=seed)

In [47]:
p_val = np.zeros((trn.shape[0], n_class))
p_tst = np.zeros((tst.shape[0], n_class))
for i, (i_trn, i_val) in enumerate(cv.split(trn, y), 1):
    print(f'training model for CV #{i}')
    clf = lgb.LGBMClassifier(**params)
    clf.fit(trn[i_trn], y[i_trn],
            eval_set=[(trn[i_val], y[i_val])],
            eval_metric='multiclass',
            early_stopping_rounds=10)
    
    p_val[i_val, :] = clf.predict_proba(trn[i_val])
    p_tst += clf.predict_proba(tst) / n_fold

training model for CV #1
[1]	valid_0's multi_logloss: 1.46148
Training until validation scores don't improve for 10 rounds.
[2]	valid_0's multi_logloss: 1.36982
[3]	valid_0's multi_logloss: 1.28951
[4]	valid_0's multi_logloss: 1.21966
[5]	valid_0's multi_logloss: 1.1572
[6]	valid_0's multi_logloss: 1.10142
[7]	valid_0's multi_logloss: 1.05099
[8]	valid_0's multi_logloss: 1.00533
[9]	valid_0's multi_logloss: 0.963745
[10]	valid_0's multi_logloss: 0.925587
[11]	valid_0's multi_logloss: 0.890479
[12]	valid_0's multi_logloss: 0.858205
[13]	valid_0's multi_logloss: 0.82865
[14]	valid_0's multi_logloss: 0.800991
[15]	valid_0's multi_logloss: 0.775566
[16]	valid_0's multi_logloss: 0.752285
[17]	valid_0's multi_logloss: 0.730716
[18]	valid_0's multi_logloss: 0.710639
[19]	valid_0's multi_logloss: 0.692192
[20]	valid_0's multi_logloss: 0.674969
[21]	valid_0's multi_logloss: 0.658876
[22]	valid_0's multi_logloss: 0.643888
[23]	valid_0's multi_logloss: 0.629926
[24]	valid_0's multi_logloss: 0.617

In [48]:
print(f'{accuracy_score(y, np.argmax(p_val, axis=1)) * 100:.4f}%')

84.7173%


In [49]:
print(p_val.shape, p_tst.shape)

(54879, 5) (19617, 5)


In [50]:
np.savetxt(p_val_file, p_val, fmt='%.6f', delimiter=',')
np.savetxt(p_tst_file, p_tst, fmt='%.6f', delimiter=',')

In [51]:
sub = pd.read_csv(sample_file, index_col=0)
print(sub.shape)
sub.head()

(19617, 5)


,0,1,2,3,4
index,,,,,
0,0,0,0,0,0
1,0,0,0,0,0
2,0,0,0,0,0
3,0,0,0,0,0
4,0,0,0,0,0


In [52]:
sub[sub.columns] = p_tst
sub.head()


,0,1,2,3,4
index,,,,,
0,0.0076,0.6127,0.3644,0.0140,0.0013
1,0.0113,0.9537,0.0034,0.0100,0.0217
2,0.9956,0.0015,0.0011,0.0005,0.0012
3,0.0056,0.0206,0.9669,0.0038,0.0031
4,0.7934,0.0201,0.0480,0.0874,0.0511


In [53]:
sub.to_csv(sub_file)